# Data Exploration and Pre Processing

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
df = pd.read_csv('C:\\Users\\bunny\\Desktop\\DAAPROJECT\\fifadata_csv.csv',delimiter = ',')
#df = open('C:\\Users\\bunny\\Desktop\\DAAPROJECT\\fifadata_csv.csv','rb').read().splitlines()
#d = df.read()
df = df.drop('Date',1)
df = df.drop('Referee',1)
#df = df.drop('HTR',1)
df = df.drop('FTHG',1)
df = df.drop('FTAG',1)

#df['FTR'] = pd.to_numeric(df['FTR'])
df1=pd.get_dummies(df).astype(np.float)
#df['FTR'] = df['FTR'].astype(object)
#c = df1.dtypes
#print(c)

#FTR = {'H': 1,'A': 2,'D': 3}
#df1.FTR=[FTR[item] for item in df1.FTR] 
#df1['FTR'] = pd.to_numeric(df1['FTR'])
#c = df1.dtypes
#print(c)
df1 = df1.drop('FTR_A',1)
df1 = df1.drop('FTR_D',1)    
train,test = df1.iloc[:3048,],df1.iloc[3048:,]
X=train.drop('FTR_H',1)
Y=train['FTR_H']
X_test=test.drop('FTR_H',1)
Y_test=test['FTR_H']
#print(Y)
print(train.head())
#print(test.head())
#df.to_csv(r'C:\\Users\\bunny\\Desktop\\DAAPROJECT\File_Name.csv')

   HTHG  HTAG    HS    AS   HST   AST    HF    AF    HC   AC  ...    \
0   0.0   1.0  11.0  14.0   5.0   7.0  15.0  14.0   4.0  6.0  ...     
1   0.0   1.0  17.0   8.0   9.0   5.0  12.0   9.0   5.0  4.0  ...     
2   0.0   1.0  11.0  20.0   3.0  13.0  16.0  10.0   4.0  7.0  ...     
3   1.0   1.0  26.0   7.0  12.0   3.0  13.0  15.0  12.0  4.0  ...     
4   0.0   3.0   8.0  15.0   5.0   9.0  11.0  13.0   4.0  9.0  ...     

   AwayTeam_Tottenham  AwayTeam_Watford  AwayTeam_West Brom  \
0                 0.0               0.0                 0.0   
1                 0.0               0.0                 0.0   
2                 0.0               0.0                 0.0   
3                 0.0               0.0                 0.0   
4                 0.0               0.0                 0.0   

   AwayTeam_West Ham  AwayTeam_Wigan  AwayTeam_Wolves  FTR_H  HTR_A  HTR_D  \
0                0.0             1.0              0.0    0.0    1.0    0.0   
1                0.0             0.0  

# Logistic Regression Manual Code

In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


def sigmoid(scores):
    return 1 / (1 + np.exp(-scores))

def logistic_regression(features, target, num_steps, learning_rate, add_intercept = False):
    if add_intercept:
        intercept = np.ones((features.shape[0], 1))
        features = np.hstack((intercept, features))
        
    weights = np.zeros(features.shape[1])
    
    for step in range(num_steps):
        scores = np.dot(features, weights)
        predictions = sigmoid(scores)

        # Update weights with gradient
        output_error_signal = target - predictions
        gradient = np.dot(features.T, output_error_signal)
        weights += learning_rate * gradient
        
    return weights


In [3]:
weights = logistic_regression(X, Y,
                     num_steps = 500, learning_rate = 0.1, add_intercept=True)
#print(weights)
data_with_intercept = np.hstack((np.ones((X_test.shape[0], 1)),
                                 X_test))
final_scores = np.dot(data_with_intercept, weights)
preds = np.round(sigmoid(final_scores))
preds
#print format((preds == Y_test).sum().astype(float) / len(preds))
from sklearn.metrics import classification_report
print(classification_report(Y_test,preds))

C:\Users\bunny\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: overflow encountered in exp
  import sys


             precision    recall  f1-score   support

        0.0       0.92      0.49      0.64       347
        1.0       0.61      0.95      0.74       294

avg / total       0.78      0.70      0.69       641



# Logistic regression from sklearn package

In [4]:
from sklearn.linear_model import LogisticRegression
logmodel = LogisticRegression()
logmodel.fit(X,Y)
predictions = logmodel.predict(X_test)

In [5]:
from sklearn.metrics import classification_report
print(classification_report(Y_test,predictions))

             precision    recall  f1-score   support

        0.0       0.78      0.86      0.82       347
        1.0       0.81      0.71      0.75       294

avg / total       0.79      0.79      0.79       641



# KNN from Scratch

In [6]:
from __future__ import print_function, division
import numpy as np
import math
#from mlfromscratch.utils import euclidean_distance
def euclidean_distance(x1, x2):
    """ Calculates the l2 distance between two vectors """
    distance = 0
    # Squared distance between each coordinate
    for i in range(len(x1)):
        distance += pow(float((float(x1[i]) - float(x2[i]))), 2)
    return math.sqrt(distance)
class KNN():
    """ K Nearest Neighbors classifier.
    Parameters:
    -----------
    k: int
        The number of closest neighbors that will determine the class of the 
        sample that we wish to predict.
    """
    def __init__(self, k=5):
        self.k = k

    def _vote(self, neighbor_labels):
        """ Return the most common class among the neighbor samples """
        counts = np.bincount(neighbor_labels.astype('int'))
        return counts.argmax()

    def predict(self, testing, X_train, y_train):
        y_pred = np.empty(testing.shape[0])
        # Determine the class of each sample
        #print((X_test))
        for i, test_sample in enumerate(testing):
            #print(i,test_sample)
            # Sort the training samples by their distance to the test sample and get the K nearest
            idx = np.argsort([euclidean_distance(test_sample, x) for x in X_train])[:self.k]
            # Extract the labels of the K nearest neighboring training samples
            k_nearest_neighbors = np.array([y_train[i] for i in idx])
            # Label sample as the most common class label
            y_pred[i] = self._vote(k_nearest_neighbors)
        #return 0
        return y_pred

In [7]:
#knn_model=KNN(k=5)
dataX=X.values
dataY=Y.values
testdataX=X_test.values
testdataY=Y_test.values
#print(dataX,dataY)
clf = KNN(k=5)
y_pred = clf.predict(testdataX, dataX, dataY)

In [8]:
from sklearn.metrics import classification_report
print(classification_report(Y_test,y_pred))

             precision    recall  f1-score   support

        0.0       0.70      0.76      0.73       347
        1.0       0.68      0.62      0.65       294

avg / total       0.69      0.69      0.69       641



# Knn from sklearn package

In [9]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X,Y)
y_skpred=knn.predict(X_test)
print(classification_report(Y_test,y_skpred))

             precision    recall  f1-score   support

        0.0       0.70      0.76      0.73       347
        1.0       0.68      0.63      0.65       294

avg / total       0.69      0.70      0.69       641



# Naive Bayes from Scratch

In [10]:
import numpy as np
import math
class NaiveBayes():
    """The Gaussian Naive Bayes classifier. """
    def fit(self, X, y):
        self.X, self.y = X, y
        self.classes = np.unique(y)
        self.parameters = []
        # Calculate the mean and variance of each feature for each class
        for i, c in enumerate(self.classes):
            # Only select the rows where the label equals the given class
            X_where_c = X[np.where(y == c)]
            self.parameters.append([])
            # Add the mean and variance for each feature (column)
            for col in X_where_c.T:
                parameters = {"mean": col.mean(), "var": col.var()}
                self.parameters[i].append(parameters)

    def _calculate_likelihood(self, mean, var, x):
        """ Gaussian likelihood of the data x given mean and var """
        eps = 1e-4 # Added in denominator to prevent division by zero
        coeff = 1.0 / math.sqrt(2.0 * math.pi * var + eps)
        exponent = math.exp(-(math.pow(x - mean, 2) / (2 * var + eps)))
        return coeff * exponent

    def _calculate_prior(self, c):
        """ Calculate the prior of class c
        (samples where class == c / total number of samples)"""
        frequency = np.mean(self.y == c)
        return frequency

    def _classify(self, sample):
        """ Classification using Bayes Rule P(Y|X) = P(X|Y)*P(Y)/P(X),
            or Posterior = Likelihood * Prior / Scaling Factor
        P(Y|X) - The posterior is the probability that sample x is of class y given the
                 feature values of x being distributed according to distribution of y and the prior.
        P(X|Y) - Likelihood of data X given class distribution Y.
                 Gaussian distribution (given by _calculate_likelihood)
        P(Y)   - Prior (given by _calculate_prior)
        P(X)   - Scales the posterior to make it a proper probability distribution.
                 This term is ignored in this implementation since it doesn't affect
                 which class distribution the sample is most likely to belong to.
        Classifies the sample as the class that results in the largest P(Y|X) (posterior)
        """
        posteriors = []
        # Go through list of classes
        for i, c in enumerate(self.classes):
            # Initialize posterior as prior
            posterior = self._calculate_prior(c)
            # Naive assumption (independence):
            # P(x1,x2,x3|Y) = P(x1|Y)*P(x2|Y)*P(x3|Y)
            # Posterior is product of prior and likelihoods (ignoring scaling factor)
            for feature_value, params in zip(sample, self.parameters[i]):
                # Likelihood of feature value given distribution of feature values given y
                likelihood = self._calculate_likelihood(params["mean"], params["var"], feature_value)
                posterior *= likelihood
            posteriors.append(posterior)
        # Return the class with the largest posterior probability
        return self.classes[np.argmax(posteriors)]

    def predict(self, X):
        """ Predict the class labels of the samples in X """
        y_pred = [self._classify(sample) for sample in X]
        return y_pred

In [11]:
#knn_model=KNN(k=5)
dataX=X.values
dataY=Y.values
testdataX=X_test.values
testdataY=Y_test.values
#print(dataX,dataY)
clf = NaiveBayes()
clf.fit(dataX,dataY)
y_NBpred = clf.predict(testdataX)
print(classification_report(Y_test,y_NBpred))

             precision    recall  f1-score   support

        0.0       0.68      0.70      0.69       347
        1.0       0.63      0.61      0.62       294

avg / total       0.66      0.66      0.66       641



# Naive Bayes from sklearn Package

In [12]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
y_SKNBpred = gnb.fit(X, Y).predict(X_test)
print(classification_report(Y_test,y_SKNBpred))

             precision    recall  f1-score   support

        0.0       0.72      0.58      0.64       347
        1.0       0.60      0.74      0.66       294

avg / total       0.67      0.65      0.65       641



# Decision tree from Scratch

In [13]:
import numpy as np
def standardize(X):
    """ Standardize the dataset X """
    X_std = X
    mean = X.mean(axis=0)
    std = X.std(axis=0)
    for col in range(np.shape(X)[1]):
        if std[col]:
            X_std[:, col] = (X_std[:, col] - mean[col]) / std[col]
    # X_std = (X - X.mean(axis=0)) / X.std(axis=0)
    return X_std

def mean_squared_error(y_true, y_pred):
    """ Returns the mean squared error between y_true and y_pred """
    mse = np.mean(np.power(y_true - y_pred, 2))
    return mse
def calculate_entropy(y):
    """ Calculate the entropy of label array y """
    log2 = lambda x: math.log(x) / math.log(2)
    unique_labels = np.unique(y)
    entropy = 0
    for label in unique_labels:
        count = len(y[y == label])
        p = count / len(y)
        entropy += -p * log2(p)
    return entropy
def calculate_variance(X):
    """ Return the variance of the features in dataset X """
    mean = np.ones(np.shape(X)) * X.mean(0)
    n_samples = np.shape(X)[0]
    variance = (1 / n_samples) * np.diag((X - mean).T.dot(X - mean))
    
    return variance

def divide_on_feature(X, feature_i, threshold):
    """ Divide dataset based on if sample value on feature index is larger than
        the given threshold """
    split_func = None
    if isinstance(threshold, int) or isinstance(threshold, float):
        split_func = lambda sample: sample[feature_i] >= threshold
    else:
        split_func = lambda sample: sample[feature_i] == threshold

    X_1 = np.array([sample for sample in X if split_func(sample)])
    X_2 = np.array([sample for sample in X if not split_func(sample)])

    return np.array([X_1, X_2])


class DecisionNode():
    """Class that represents a decision node or leaf in the decision tree
    Parameters:
    -----------
    feature_i: int
        Feature index which we want to use as the threshold measure.
    threshold: float
        The value that we will compare feature values at feature_i against to
        determine the prediction.
    value: float
        The class prediction if classification tree, or float value if regression tree.
    true_branch: DecisionNode
        Next decision node for samples where features value met the threshold.
    false_branch: DecisionNode
        Next decision node for samples where features value did not meet the threshold.
    """
    def __init__(self, feature_i=None, threshold=None,
                 value=None, true_branch=None, false_branch=None):
        self.feature_i = feature_i          # Index for the feature that is tested
        self.threshold = threshold          # Threshold value for feature
        self.value = value                  # Value if the node is a leaf in the tree
        self.true_branch = true_branch      # 'Left' subtree
        self.false_branch = false_branch    # 'Right' subtree


# Super class of RegressionTree and ClassificationTree
class DecisionTree(object):
    """Super class of RegressionTree and ClassificationTree.
    Parameters:
    -----------
    min_samples_split: int
        The minimum number of samples needed to make a split when building a tree.
    min_impurity: float
        The minimum impurity required to split the tree further.
    max_depth: int
        The maximum depth of a tree.
    loss: function
        Loss function that is used for Gradient Boosting models to calculate impurity.
    """
    def __init__(self, min_samples_split=2, min_impurity=1e-7,
                 max_depth=float("inf"), loss=None):
        self.root = None  # Root node in dec. tree
        # Minimum n of samples to justify split
        self.min_samples_split = min_samples_split
        # The minimum impurity to justify split
        self.min_impurity = min_impurity
        # The maximum depth to grow the tree to
        self.max_depth = max_depth
        # Function to calculate impurity (classif.=>info gain, regr=>variance reduct.)
        self._impurity_calculation = None
        # Function to determine prediction of y at leaf
        self._leaf_value_calculation = None
        # If y is one-hot encoded (multi-dim) or not (one-dim)
        self.one_dim = None
        # If Gradient Boost
        self.loss = loss

    def fit(self, X, y, loss=None):
        """ Build decision tree """
        self.one_dim = len(np.shape(y)) == 1
        self.root = self._build_tree(X, y)
        self.loss=None

    def _build_tree(self, X, y, current_depth=0):
        """ Recursive method which builds out the decision tree and splits X and respective y
        on the feature of X which (based on impurity) best separates the data"""

        largest_impurity = 0
        best_criteria = None    # Feature index and threshold
        best_sets = None        # Subsets of the data

        # Check if expansion of y is needed
        if len(np.shape(y)) == 1:
            y = np.expand_dims(y, axis=1)

        # Add y as last column of X
        Xy = np.concatenate((X, y), axis=1)

        n_samples, n_features = np.shape(X)

        if n_samples >= self.min_samples_split and current_depth <= self.max_depth:
            # Calculate the impurity for each feature
            for feature_i in range(n_features):
                # All values of feature_i
                feature_values = np.expand_dims(X[:, feature_i], axis=1)
                unique_values = np.unique(feature_values)

                # Iterate through all unique values of feature column i and
                # calculate the impurity
                for threshold in unique_values:
                    # Divide X and y depending on if the feature value of X at index feature_i
                    # meets the threshold
                    Xy1, Xy2 = divide_on_feature(Xy, feature_i, threshold)

                    if len(Xy1) > 0 and len(Xy2) > 0:
                        # Select the y-values of the two sets
                        y1 = Xy1[:, n_features:]
                        y2 = Xy2[:, n_features:]

                        # Calculate impurity
                        impurity = self._impurity_calculation(y, y1, y2)

                        # If this threshold resulted in a higher information gain than previously
                        # recorded save the threshold value and the feature
                        # index
                        if impurity > largest_impurity:
                            largest_impurity = impurity
                            best_criteria = {"feature_i": feature_i, "threshold": threshold}
                            best_sets = {
                                "leftX": Xy1[:, :n_features],   # X of left subtree
                                "lefty": Xy1[:, n_features:],   # y of left subtree
                                "rightX": Xy2[:, :n_features],  # X of right subtree
                                "righty": Xy2[:, n_features:]   # y of right subtree
                                }

        if largest_impurity > self.min_impurity:
            # Build subtrees for the right and left branches
            true_branch = self._build_tree(best_sets["leftX"], best_sets["lefty"], current_depth + 1)
            false_branch = self._build_tree(best_sets["rightX"], best_sets["righty"], current_depth + 1)
            return DecisionNode(feature_i=best_criteria["feature_i"], threshold=best_criteria[
                                "threshold"], true_branch=true_branch, false_branch=false_branch)

        # We're at leaf => determine value
        leaf_value = self._leaf_value_calculation(y)

        return DecisionNode(value=leaf_value)


    def predict_value(self, x, tree=None):
        """ Do a recursive search down the tree and make a prediction of the data sample by the
            value of the leaf that we end up at """

        if tree is None:
            tree = self.root

        # If we have a value (i.e we're at a leaf) => return value as the prediction
        if tree.value is not None:
            return tree.value

        # Choose the feature that we will test
        feature_value = x[tree.feature_i]

        # Determine if we will follow left or right branch
        branch = tree.false_branch
        if isinstance(feature_value, int) or isinstance(feature_value, float):
            if feature_value >= tree.threshold:
                branch = tree.true_branch
        elif feature_value == tree.threshold:
            branch = tree.true_branch

        # Test subtree
        return self.predict_value(x, branch)

    def predict(self, X):
        """ Classify samples one by one and return the set of labels """
        y_pred = [self.predict_value(sample) for sample in X]
        return y_pred

    def print_tree(self, tree=None, indent=" "):
        """ Recursively print the decision tree """
        if not tree:
            tree = self.root

        # If we're at leaf => print the label
        if tree.value is not None:
            print (tree.value)
        # Go deeper down the tree
        else:
            # Print test
            print ("%s:%s? " % (tree.feature_i, tree.threshold))
            # Print the true scenario
            print ("%sT->" % (indent), end="")
            self.print_tree(tree.true_branch, indent + indent)
            # Print the false scenario
            print ("%sF->" % (indent), end="")
            self.print_tree(tree.false_branch, indent + indent)




class ClassificationTree(DecisionTree):
    def _calculate_information_gain(self, y, y1, y2):
        # Calculate information gain
        p = len(y1) / len(y)
        entropy = calculate_entropy(y)
        info_gain = entropy - p * \
            calculate_entropy(y1) - (1 - p) * \
            calculate_entropy(y2)

        return info_gain

    def _majority_vote(self, y):
        most_common = None
        max_count = 0
        for label in np.unique(y):
            # Count number of occurences of samples with label
            count = len(y[y == label])
            if count > max_count:
                most_common = label
                max_count = count
        return most_common

    def fit(self, X, y):
        self._impurity_calculation = self._calculate_information_gain
        self._leaf_value_calculation = self._majority_vote
        super(ClassificationTree, self).fit(X, y)

In [14]:
dataX=X.values
dataY=Y.values
testdataX=X_test.values
testdataY=Y_test.values
#print(dataX,dataY)
clf = ClassificationTree()
clf.fit(dataX, dataY)
y_DTpred = clf.predict(testdataX)
print(classification_report(Y_test,y_DTpred))

             precision    recall  f1-score   support

        0.0       0.72      0.76      0.74       347
        1.0       0.70      0.65      0.67       294

avg / total       0.71      0.71      0.71       641



# Classification tree from sklearn

In [15]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X, Y)
y_DTSKpred=clf.predict(X_test)
print(classification_report(Y_test,y_DTSKpred))

             precision    recall  f1-score   support

        0.0       0.72      0.76      0.74       347
        1.0       0.69      0.65      0.67       294

avg / total       0.71      0.71      0.71       641



# Ada Boost from sklearn package

In [16]:
from sklearn.ensemble import AdaBoostClassifier
# Create adaboost classifer object
abc = AdaBoostClassifier(n_estimators=30,
                         learning_rate=0.4)
# Train Adaboost Classifer
model = abc.fit(X, Y)

#Predict the response for test dataset
y_adaSKpred = model.predict(X_test)
print(classification_report(Y_test,y_adaSKpred))

C:\Users\bunny\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


             precision    recall  f1-score   support

        0.0       0.76      0.88      0.81       347
        1.0       0.82      0.67      0.74       294

avg / total       0.79      0.78      0.78       641



# XGBoost from package

In [17]:
import xgboost as xgb
model = xgb.XGBClassifier(n_estimators=50, max_depth=2, learning_rate=0.2, subsample=0.5)
dataX=X.values
dataY=Y.values
testdataX=X_test.values
testdataY=Y_test.values
train_model = model.fit(dataX, dataY)
xgpred = train_model.predict(testdataX)
print(classification_report(Y_test,xgpred))

             precision    recall  f1-score   support

        0.0       0.76      0.87      0.81       347
        1.0       0.81      0.68      0.74       294

avg / total       0.79      0.78      0.78       641



C:\Users\bunny\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


# SVM from package

In [18]:
from sklearn import svm
from sklearn.svm import SVC
dataX=X.values
dataY=Y.values
testdataX=X_test.values
testdataY=Y_test.values
svclassifier = SVC(kernel='linear')  
svclassifier.fit(dataX, dataY)
y_pred = svclassifier.predict(testdataX)
print(classification_report(Y_test,y_pred))

             precision    recall  f1-score   support

        0.0       0.75      0.88      0.81       347
        1.0       0.82      0.65      0.73       294

avg / total       0.78      0.78      0.77       641



# Deep neural Network

In [19]:
from keras.models import Sequential
from keras.layers import Dense
dataX=X.values
dataY=Y.values
testdataX=X_test.values
testdataY=Y_test.values
model = Sequential()
model.add(Dense(12, input_dim=89, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.


In [20]:
model.fit(dataX, dataY, epochs=20, batch_size=10)
scores = model.evaluate(testdataX,Y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

Instructions for updating:
Use tf.cast instead.
Epoch 1/20
3048/3048 [==============================] - 3s 880us/step - loss: 0.6103 - acc: 0.6673
Epoch 2/20
3048/3048 [==============================] - 1s 195us/step - loss: 0.4748 - acc: 0.7730
Epoch 3/20
3048/3048 [==============================] - 1s 198us/step - loss: 0.4572 - acc: 0.7792
Epoch 4/20
3048/3048 [==============================] - 1s 192us/step - loss: 0.4476 - acc: 0.7890
Epoch 5/20
3048/3048 [==============================] - 1s 196us/step - loss: 0.4435 - acc: 0.7913
Epoch 6/20
3048/3048 [==============================] - 1s 243us/step - loss: 0.4434 - acc: 0.7943
Epoch 7/20
3048/3048 [==============================] - 1s 222us/step - loss: 0.4389 - acc: 0.7940
Epoch 8/20
3048/3048 [==============================] - 1s 201us/step - loss: 0.4412 - acc: 0.8025
Epoch 9/20
3048/3048 [==============================] - 1s 198us/step - loss: 0.4385 - acc: 0.7956
Epoch 10/20
3048/3048 [==============================] - 1s 1